In [7]:
# First we will do a stochastic slip distribution
import geostochpy
from geostochpy import modcomcot
import numpy as np
from scipy.io import savemat,loadmat

In [8]:
comcot=geostochpy.get_data('comcot.ctl')
with open(comcot, 'r') as file:
    lines=file.readlines()

In [9]:
nx=18
ny=54
width=180
length=540
dx,dy=width/nx,length/ny
route_trench = "../../auxiliar/trench-chile.txt"
lons_fosa, lats_fosa  = geostochpy.load_trench(route_trench)
# load slab files
slabdep,slabdip,slabstrike,slabrake=geostochpy.load_files_slab2(zone='south_america',rake=True)
north=-29.5
lons,lons_ep,lats,lats_ep=geostochpy.make_fault_alongtrench(lons_fosa,lats_fosa,north, nx,ny,width,length)
lons_ep2 = [x + 360 for x in lons_ep]
[X_grid,Y_grid,dep,dip,strike,rake]=geostochpy.interp_slabtofault(lons,lats,nx,ny,slabdep,slabdip,slabstrike,slabrake)
## Creation slip models
# mean matrix
#
Mw=9.0
media=geostochpy.media_slip(Mw,length*1000,width*1000,dep)
leveque_taper=geostochpy.taper_LeVeque(dep,55000)
# leveque_taper=leveque_taper/np.max(leveque_taper)
villarroel_taper=geostochpy.taper_except_trench_tukey(dep,alpha_dip=0.3,alpha_strike=0.3)
taper=leveque_taper*villarroel_taper
# taper=geostochpy.taper_except_trench_tukey(dep,alpha_dip=0.6,alpha_strike=0.4,dip_taperfunc=geostochpy.taper_LeVeque,strike_taperfunc=geostochpy.tukey_window_equal)

mu = geostochpy.matriz_medias_villarroel(media,taper)
# matriz de covarianza
C    = geostochpy.matriz_covarianza_optimized(dip, dep, X_grid, Y_grid,length*1000,width*1000)
# for comcot simulation
Slip=geostochpy.distribucion_slip(C, mu, 20)
Slip=geostochpy.escalar_magnitud_momento(Mw, Slip, dep, X_grid, Y_grid,prem=True) # se escala el Slip a la magnitud deseada <--------- Slip final
# Hypocenter=geostochpy.hypocenter(X_grid,Y_grid,dep,length,width) se tiene en cuenta la rigidez con el modelo PREM incluido @fetched with Rockhound

In [10]:
mdic_multifault={'depth':dep.flat,'length':dy*np.ones((1,nx*ny)),'width':dx*np.ones((1,nx*ny)),
              'slip':Slip.flat,'strike':strike.flat,'dip':dip.flat,'rake':rake.flat
              ,'lat':lats_ep,'lon':lons_ep2,'time':np.zeros((1,nx*ny))}
savemat('simulation'+'.mat',mdic_multifault)
list=modcomcot.make_multifault('simulation.mat','fault_multi.ctl','global_salida.xyz')



In [11]:
modcomcot.make_comcot_ctl(3600,60,nx*ny,list,['global_salida.xyz','valpo4_salida.xyz'])

In [12]:
print(lons_ep2)

[287.68086351969106, 287.7708725205912, 287.86088152149125, 287.9508905223913, 288.04089952329144, 288.1309085241915, 288.22091752509164, 288.3109265259917, 288.4009355268918, 288.4909445277919, 288.58095352869196, 288.6709625295921, 288.76097153049216, 288.8509805313922, 288.94098953229235, 289.0309985331924, 289.12100753409254, 289.2110165349926, 287.65765345585714, 287.74766245675727, 287.83767145765734, 287.92768045855746, 288.01768945945753, 288.1076984603576, 288.1977074612577, 288.2877164621578, 288.3777254630579, 288.467734463958, 288.55774346485805, 288.6477524657582, 288.73776146665824, 288.82777046755837, 288.91777946845843, 289.0077884693585, 289.0977974702586, 289.1878064711587, 287.6344433920233, 287.72445239292335, 287.8144613938234, 287.90447039472355, 287.9944793956236, 288.08448839652374, 288.1744973974238, 288.2645063983239, 288.354515399224, 288.44452440012407, 288.5345334010242, 288.62454240192426, 288.7145514028243, 288.80456040372445, 288.8945694046245, 288.98457